In [ ]:
import jieba
import json
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import regex as re
from string import punctuation
import xml.etree.ElementTree as ET

# Read XML File

In [ ]:
tree = ET.parse('Xdf/dataset/nus-sms-corpus-master/smsCorpus_zh_2015.03.09.xml')
root = tree.getroot()

In [ ]:
raw = pd.DataFrame(columns=['id', 'text', 'label'])
count = 0
empty = []
for tag in root.findall('message/text'):
    
    if count%1000 == 0:
        print(count)
        
    text_phrase = ET.tostring(tag, encoding='utf8', method='xml').decode('UTF-8', 'strict')[44:-7]
    text_phrase = text_phrase.replace(' ','')
    sentence = []

    tem = re.split(r'\s+|[?？]\s*', text_phrase)

    #loops through list after question mark
    for i in range(len(tem)):
        #add question mark to every element
        if tem[i] != tem[-1]:
            tem[i] += '?'
        sentence += list(filter(lambda x: x != "?" and x != "", re.split(r'\s+|[.。!！]\s*', tem[i])))

    for sen in sentence:
        if '？' in sen or '?' in sen:
            raw.loc[count] = [count, re.sub(r'\s+|[，,]\s*', '', sen[:-1]), 1]
        else:
            raw.loc[count] = [count, re.sub(r'\s+|[，,]\s*', '', sen), 0]

        count += 1

In [ ]:
raw.to_csv('nusconv_nopunc.csv')

# Word Segmentation and Stopword Removal

In [ ]:
processed = pd.read_csv('rawdata/nusconv_nopunc.csv')
processed.drop([38848], inplace=True)
stopwords = pd.read_csv('cn_stopwords.txt').iloc[:, 0].to_list()

list_of_text = processed['text'].to_list()

for i in range(len(list_of_text)):
    segmented = ','.join(jieba.cut(list_of_text[i]))
    segmented = segmented.split(',')
    rsw = ''
    for word in segmented:
        if word not in stopwords:
            rsw += word
            rsw += ' '
    processed.loc[i, 'text'] = rsw

processed = processed[['text', 'label']]
processed.loc[38848, 'label'] = 0.0
print(processed)

# Feature selection

In [ ]:
''' 
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = processed['text'].to_list()

vectorizer = TfidfVectorizer()
X =  vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names)
print(X)
'''

In [ ]:
def get_words():
    '''
    doc is each document in numpy string format
    corpus is a list of sentences each contained in a seperate list
    '''
    corpus = []
    
    for doc_num in range(100):
        if doc_num < 10:
            doc = np.genfromtxt('rawdata/wikipedia/wiki_0' + str(doc_num) + '.txt', dtype=str, delimiter='\n')
        elif doc_num >= 10 and doc_num < 33:
            doc = np.genfromtxt('rawdata/wikipedia/wiki_' + str(doc_num) + '.txt', dtype=str, delimiter='\n')
        elif doc_num >= 33:
            doc = np.genfromtxt('rawdata/wikipedia/wiki_' + str(doc_num), dtype=str, delimiter='\n')
        for i in range(len(doc)):
            try:
                for j in re.split(r'\s+|[.。!！?？]\s*', json.loads(doc[i])['text']):
                    corpus += [list(jieba.cut(re.sub(r'\s+|[，,、]\s*', '', j)))]
            except:
                pass

    return corpus

In [ ]:
import gensim

senten = processed['text'].to_list()
spl = [list(i.split()) for i in senten] + get_words()
model = gensim.models.Word2Vec(spl, min_count=1)

In [ ]:
processed_original = pd.read_csv('rawdata/nusconv_nopunc.csv')

In [ ]:
empty = [0.0] * 100
'''
lst = [not np.array_equal(vectorized.text[i], empty) for i in range(len(vectorized))]
check = pd.DataFrame(lst, columns=['text'])

vectorized_rm = vectorized[check]
'''

processed = processed[processed.text != '']
processed.reset_index(inplace=True)


print(processed)

In [ ]:
#not efficient

processed_originalcut = pd.DataFrame(columns=['text', 'label'])

processed_originalcut = processed_original[processed_original.id.isin(processed['index'].to_list())]

processed_originalcut.reset_index(inplace = True)


'''
count = 0

for i in processed['index']:
    processed_originalcut.loc[count, :] = processed_original.loc[i, :]
    count += 1
'''

print(processed_originalcut)

In [ ]:
vectorized = pd.DataFrame(columns=['text', 'label'])
count = 0
for i in processed['text']:
    vec = np.zeros([1, 100])
    for j in list(i.split()):
        vec += model[j]
    vectorized.loc[count, 'text'] = vec[0]
    vectorized.loc[count, 'label'] = processed.loc[count, 'label']
    count += 1
    
print(vectorized.head())

In [ ]:
model.most_similar(positive=['不', '没有'], topn=5)

In [ ]:
pos = vectorized.loc[vectorized['label'] == 1.0]
pos_text = processed.loc[vectorized['label'] == 1.0]
pto = processed_originalcut.loc[vectorized['label'] == 1.0]

neg = vectorized.loc[vectorized['label'] == 0.0][0:6000]
neg_text = processed.loc[vectorized['label'] == 0.0][0:6000]
nto = processed_originalcut.loc[vectorized['label'] == 0.0][0:6000]

resampled = pos.append(neg)
resampled_text = pos_text.append(neg_text)
rto = pto.append(nto)

shuffled = resampled.sample(n=12000, random_state=1)
shuffled_text = resampled_text.sample(n=12000, random_state=1)
sto = rto.sample(n=12000, random_state=1)

shuffled.reset_index(inplace = True)
shuffled_text.reset_index(inplace = True)
sto.reset_index(inplace=True)

x_train = shuffled.loc[0:10000, 'text'].to_list()
y_train = shuffled.loc[0:10000, 'label'].to_list()

x_test = shuffled.loc[10000:, 'text'].to_list()
y_test = shuffled.loc[10000:, 'label'].to_list()

xtt = shuffled_text.loc[10000:, 'text'].to_list()

xot = sto.loc[10000:, 'text'].to_list()

# Train

In [ ]:
from sklearn import svm
from sklearn.linear_model import LinearRegression


clf = svm.SVC()
clf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve

y_pred = clf.predict(x_test)
fpr, tpr, threshhold = roc_curve(y_test, y_pred, pos_label=2)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("auc: ", auc(fpr, tpr))

#floor = c < 0.5
#c[floor] = 0
#ceiling = c >= 0.5
#c[ceiling] = 1

print(y_pred)

# Plotting

In [ ]:
print(clf.predict((model['']).reshape(1, -1)))

In [ ]:
wrong = pd.DataFrame(columns=["Index", "Original_Text", "Processed_Text", "Predicted", "Labels"])

count = 0
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        wrong.loc[count, :] = i, xot[i], xtt[i].strip(), y_pred[i], y_test[i]
        count += 1
        #print('Index: {},\nText: {},\nPredicted: {},\nLabel: {}\n'.format(i, xtt[i].strip(), y_pred[i], y_test[i]))


print(wrong.loc[570:590, :])
wrong.to_csv('wrong3.csv')

In [ ]:
from sklearn.metrics import plot_roc_curve, confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt

confusion_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
print(confusion_matrix)
disp = plot_confusion_matrix(clf, x_test, y_test,
                                 display_labels=['Non Question','Question'],
                                 cmap=plt.cm.Blues,
                                 normalize='true')
disp.ax_.set_title('confusion_matrix')

plt.show()

print(disp.confusion_matrix)

In [ ]:
ax = plt.gca()
#clf_display = plot_roc_curve(clf, x_train, y_train, ax=ax, alpha=0.8)
svc_display = plot_roc_curve(clf, x_train, y_train)
svc_display.plot(ax=ax, alpha=0.8)

In [ ]:
processed.to_csv('nusprocessed.csv')
vectorized.to_csv('nusvectorized.csv')

In [ ]:


# create a mesh to plot in

h = 0.02

x_min, x_max = np.amin(x_train) - 1, np.amax(x_train) + 1
y_min, y_max = np.amin(y_train) - 1, np.amax(y_train) + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))


# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
plt.subplot(2, 2, 1)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.title(' ')

plt.show()